In [1]:
import pprint
import pandas as pd
from ORBIT import ProjectManager, SupplyChainManager, load_config
from ORBIT.supply_chain import DEFAULT_MULTIPLIERS, LABOR_SPLIT

pp = pprint.PrettyPrinter(indent=4)


In [2]:
# These are from the spreadsheet on teams
# They can be overridden in ORBIT/supply_chain.py or passed as a new dict
# into the manager class like SupplyChainManager(supply_chain, multipliers={different multipliers})
pp.pprint(DEFAULT_MULTIPLIERS)

{   'array_cable': {'domestic': 0.074, 'imported': 0.0},
    'blades': {'domestic': 0.026, 'imported': 0.31},
    'export_cable': {'domestic': 0.128, 'imported': 0.0},
    'monopile': {'domestic': 0.085, 'imported': 0.28, 'tariffs': 0.25},
    'nacelle': {'domestic': 0.025, 'imported': 0.07},
    'oss_substructure': {'domestic': 0.0, 'imported': 0.0},
    'oss_topside': {'domestic': 0.0, 'imported': 0.0},
    'tower': {'domestic': 0.04, 'imported': 0.12, 'tariffs': 0.25},
    'transition_piece': {'domestic': 0.169, 'imported': 0.18, 'tariffs': 0.25}}


In [3]:
# These are used for the tariff calculations: tariffs = (1 - labor_split) * total_cost * tariff_rate
# % of cost that is labor vs material
pp.pprint(LABOR_SPLIT)

{'monopile': 0.5, 'oss_topside': 0.5, 'tower': 0.5, 'transition_piece': 0.5}


In [4]:
# Load Sample Config
config = load_config("examples/configs/example_fixed_project.yaml")

In [5]:
# Define supply chain scenario to use
# Options are imported, domestic or domestic, imported steel
supply_chain = {
    "blades": "imported",
    "nacelle": "imported",
    "tower": "domestic, imported steel",
    "monopile": "domestic, imported steel",
    "transition_piece": "domestic, imported steel",
    "array_cable": "domestic",
    "export_cable": "domestic",
    "oss_topside": "domestic"
}

sc_manager = SupplyChainManager(supply_chain)

In [6]:
# Run a project through this method
# It will perform the supply chain adjustments and return a project instance with the adjusted values
sc_project = sc_manager.run_project(config)

ORBIT library intialized at '/Users/jnunemak/repos/external/ORBIT/library'


In [7]:
# This is just for comparison
project = ProjectManager(config)
project.run()

In [8]:
# Comparing the values from SupplyChainManager vs. ProjectManager
df = pd.concat([
    pd.Series(project.capex_breakdown, name="ORBIT"),
    pd.Series(sc_project.capex_breakdown, name="SupplyChain")
], axis=1)

df["Ratio"] = df["SupplyChain"] / df["ORBIT"]

In [9]:
df

,ORBIT,SupplyChain,Ratio
Array System,2.441658e+07,2.622340e+07,1.074000
Export System,2.281350e+07,2.573363e+07,1.128000
Offshore Substation,4.973955e+07,4.973955e+07,1.000000
Scour Protection,5.896000e+06,5.896000e+06,1.000000
Substructure,1.544362e+08,1.926734e+08,1.247592
Array System Installation,1.305663e+07,1.305663e+07,1.000000
Export System Installation,6.203998e+07,6.203998e+07,1.000000
Offshore Substation Installation,3.409502e+06,3.409502e+06,1.000000
Scour Protection Installation,1.285632e+07,1.285632e+07,1.000000
Substructure Installation,2.249919e+07,2.249919e+07,1.000000
